In [15]:
# 20天 
# SMA, 20天移动平均线
# EMA, 20天指数移动平均线
# MACD, 12天指数移动平均线 - 26天指数移动平均线


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import pandas_datareader.data as web
from datetime import datetime, timedelta
import scipy.stats as stats
from sklearn.metrics import brier_score_loss, roc_curve, auc, log_loss
from sklearn.preprocessing import StandardScaler

In [23]:
df = pd.read_csv('New_Rolling_Predicted_SP_lg_pr.csv')


cols = df.columns.tolist()
cols.insert(-2, cols.pop(cols.index('SP_lg_pr')))
df = df[cols]

# rename SP_lg_pr to SP_lg_pr(Time_T), Next_Week_SP_lg_pr to SP_lg_pr(Time_T+1), Predicted_Next_Week_SP_lg_pr to Predicetd_SP_lg_pr(Time_T+1)
df.rename(columns={'SP_lg_pr': 'SP_lg_pr(Time_T)', 'Next_Week_SP_lg_pr': 'SP_lg_pr(Time_T+1)', 'Predicted_Next_Week_SP_lg_pr': 'Predicted_SP_lg_pr(Time_T+1)'}, inplace=True)
df.set_index('Date', inplace=True)

# keep the last three columns
df = df.iloc[:, -3:]
df.head()


,SP_lg_pr(Time_T),SP_lg_pr(Time_T+1),Predicted_SP_lg_pr(Time_T+1)
Date,,,
2021-05-12,8.309687,8.322559,8.309902
2021-05-19,8.322559,8.341885,8.321608
2021-05-26,8.341885,8.344771,8.341365
2021-06-02,8.344771,8.347484,8.351182
2021-06-09,8.347484,8.348467,8.348923


In [26]:
Backtest = df.copy()

# read the SP500 price data
SP500 = pd.read_csv("^SPX.csv")
SP500.set_index('Date', inplace=True)

start_date = Backtest.index[0]
end_date = Backtest.index[-1]
print(start_date, end_date)

# merge SP500 price data to Backtest
Combined = Backtest.merge(SP500, how='left', left_index=True, right_index=True)
# drop columns of High, Low, Close, and Volume
Final = Combined.drop(['High', 'Low', 'Open', 'Adj Close', 'Volume'], axis=1)
Final.head()

2021-05-12 2024-01-31


,SP_lg_pr(Time_T),SP_lg_pr(Time_T+1),Predicted_SP_lg_pr(Time_T+1),Close
Date,,,,
2021-05-12,8.309687,8.322559,8.309902,4063.040039
2021-05-19,8.322559,8.341885,8.321608,4115.680176
2021-05-26,8.341885,8.344771,8.341365,4195.990234
2021-06-02,8.344771,8.347484,8.351182,4208.120117
2021-06-09,8.347484,8.348467,8.348923,4219.549805
